In [1]:
import re
import csv
import math
import json
import numpy as np
import pandas as pd
import pickle
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import lyricsgenius
genius_TOKEN = 'sD0C3epnJdfOQQK4eIC45dHl-Qv7DipToGpuj1n4WeuG5_LDP1HKn31w5Cn1lOux'
genius = lyricsgenius.Genius(genius_TOKEN)
genius.verbose = False
genius.remove_section_headers = True

In [3]:
movies = pd.read_csv('merged_data.csv')

In [4]:
def listify(df):
    genres = []
    languages = []
    countries = []
    for x,y,z in zip(df['Genres'],df['Languages'],df['Countries']):
        g = re.findall(': \"(.*?)\"', x)
        l = re.findall(': \"(.*?)\"', y)
        c = re.findall(': \"(.*?)\"', z)
        genres.append(g)
        languages.append(l)
        countries.append(c)
    df['Genres'] = genres
    df['Languages'] = languages
    df['Countries'] = countries

In [5]:
listify(movies)

In [6]:
vectorizer = TfidfVectorizer(max_features=50000, max_df=0.8, min_df=20, norm='l2')
tokenizer = vectorizer.build_tokenizer()

In [7]:
movies['toks'] = [tokenizer(summary) for summary in movies['Summary']]

In [8]:
#is_english = ['English Language' in m for m in movies['Languages']]
#movies = movies[is_english]

In [9]:
'''years = []
for x in movies['ReleaseDate']:
    x = str(x)[:4]
    if x != 'nan':
        years.append(int(x))
    else:
        years.append(3000)
movies['ReleaseDate'] = years'''

"years = []\nfor x in movies['ReleaseDate']:\n    x = str(x)[:4]\n    if x != 'nan':\n        years.append(int(x))\n    else:\n        years.append(3000)\nmovies['ReleaseDate'] = years"

In [10]:
#movies = movies[movies['ReleaseDate']>=1960]  

In [11]:
#movies = movies.reset_index()
#movies = movies.drop(['index'], axis=1)

In [12]:
movies.head()

,WikiID,Title,ReleaseDate,Runtime,Languages,Countries,Genres,Summary,compound,pos,neg,neu,toks
0,975900,Ghosts of Mars,2001-08-24,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...","Set in the second half of the 22nd century, th...",-0.9913,0.065,0.150,0.786,"[Set, in, the, second, half, of, the, 22nd, ce..."
1,9363483,White Of The Eye,1987,110.0,[English Language],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri...",A series of murders of rich young women throug...,-0.9985,0.078,0.198,0.724,"[series, of, murders, of, rich, young, women, ..."
2,261236,A Woman in Flames,1983,106.0,[German Language],[Germany],[Drama],"Eva, an upper class housewife, becomes frustra...",0.9604,0.100,0.065,0.835,"[Eva, an, upper, class, housewife, becomes, fr..."
3,18998739,The Sorcerer's Apprentice,2002,86.0,[English Language],[South Africa],"[Family Film, Fantasy, Adventure, World cinema]","Every hundred years, the evil Morgana returns...",-0.8885,0.092,0.131,0.776,"[Every, hundred, years, the, evil, Morgana, re..."
4,6631279,Little city,1997-04-04,93.0,[English Language],[United States of America],"[Romantic comedy, Ensemble Film, Comedy-drama,...","Adam, a San Francisco-based artist who works a...",-0.7097,0.089,0.108,0.804,"[Adam, San, Francisco, based, artist, who, wor..."


In [13]:
num_movies = len(movies)
num_movies

42204

In [14]:
def build_ivdidx_df(movies):
    word_set = []
    for t in movies['toks']:
        word_set+=t
    word_set = set(word_set)
    word_dict = {w: [[],[]] for w in word_set}
    for i in range(len(movies)):
        tokens = movies['toks'][i]
        for w in set(tokens):
            cw = tokens.count(w)
            word_dict[w][0].append(i)
            word_dict[w][1].append(cw)
    df = pd.DataFrame.from_dict(word_dict, orient='index')
    return df

In [15]:
idx_df = build_ivdidx_df(movies)

ValueError: Length mismatch: Expected axis has 2 elements, new values have 3 elements

In [17]:
idx_df.columns = ['docs','counts']
idx_df.head()

,docs,counts
Bacolod,[22137],[1]
cities,"[363, 711, 843, 1029, 1037, 1348, 1532, 1765, ...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
sympathising,[11878],[1]
Obliged,"[4784, 12949, 13936, 15556]","[1, 1, 1, 1]"
Nyla,"[4234, 15129, 25097, 39512]","[28, 1, 2, 2]"


In [18]:
idx_df.to_csv('inv_idx.csv')